## Post Validation - Build detection History

In [1]:
import pandas as pd
import os
from pathlib import Path
from natsort import natsorted  # Natural sorting

In [3]:
# We indicte the paths
main_dir = "/mnt/d/night_recordings_analysis/"
validation_file = os.path.join(main_dir, "Ferruginous Pygmy-Owl/fepowl_Validation.txt")
template_file = os.path.join(main_dir, "detection_history_template.csv")
output_file = os.path.join(main_dir, "Ferruginous Pygmy-Owl/fepowl_detection_history.csv")


In [4]:
# We load the data
val_df = pd.read_csv(validation_file, sep='\t')
template = pd.read_csv(template_file, index_col=0)  # Site_IDs as row index


In [5]:
# Extract Site_ID and SN from file name
def parse_site_sn(filename):
    parts = Path(filename).stem.split('_')
    try:
        site_id = parts[-2]
        sn = parts[-1].replace("sn", "").zfill(2)  # Ensures consistent "01", "02", ..., "35" formatting
        return pd.Series([site_id, sn])
    except Exception as e:
        print(f"Failed to parse filename: {filename}")
        return pd.Series([None, None])


In [6]:
# Apply parsing
val_df[['Site_ID', 'SN']] = val_df['Begin File'].apply(parse_site_sn)
val_df['SN'] = val_df['SN'].astype(str).str.zfill(2)

In [7]:
# Make sure Eval is numeric
val_df['Eval'] = pd.to_numeric(val_df['Eval'], errors='coerce').astype(int)

In [8]:
# Start from template and insert 1s where Eval == 1

detection_history = template.copy() #copy prevent altering original template

for _, row in val_df.iterrows():
    if row['Eval'] == 1:
        site = row['Site_ID']
        sn = row['SN']
        if site in detection_history.index and sn in detection_history.columns:
            detection_history.loc[site, sn] = 1  # Replace 0 with 1

In [9]:
# Natural Sort
# natsorted prevents wrong order like 1, 10, 11, ..., 2
detection_history = detection_history.reindex(natsorted(detection_history.index), axis=0)
detection_history = detection_history[natsorted(detection_history.columns)]


In [10]:
# Save
detection_history.to_csv(output_file)

print("✅ Detection history created and saved to:", output_file)

✅ Detection history created and saved to: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/fepowl_detection_history.csv
